### **Python Data Engineering Project**
##### *-- Anh Vi Pham --*

### **Setup Database**

#### Connect Server

In [ ]:
import pyodbc
import json
import pandas as pd 
# database credentials
server = 'localhost,1433' 
database = 'master' 
driver = '{ODBC Driver 18 for SQL Server}'

# import keys from json file
with open("database_keys.json", "r") as file:
    config = json.load(file)
username = config["username"]
password = config["password"]

# establish connecttion

conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={database};UID={username};PWD={password};TrustServerCertificate=yes')
cursor = conn.cursor()

#### Create new database

In [24]:
def get_current_database(cursor=cursor):
    cursor.execute("SELECT DB_NAME() AS CurrentDatabase")
    current_db = cursor.fetchone()[0]
    return print(f"Currently connected to database: {current_db}")
get_current_database(cursor)

Currently connected to database: master


In [25]:
# drop database
conn.autocommit = True

# Drop the database if it exists
cursor.execute("IF EXISTS (SELECT * FROM sys.databases WHERE name = 'HospitalOperation') DROP DATABASE HospitalOperation")
print("Database 'HospitalOperation' dropped successfully if it existed.")

Database 'HospitalOperation' dropped successfully if it existed.


In [26]:
conn.autocommit = True
try:
    cursor.execute("CREATE DATABASE HospitalOperation")
    print("Database 'HospitalOperation' created successfully.")
except Exception as e:
    print(f"Error: {e}")

conn.autocommit = False

Database 'HospitalOperation' created successfully.


#### Connect to new database

In [27]:
# Reconnect to the new database
conn.close()  # Close the current connection to the master database

new_database_name = 'HospitalOperation'
conn = pyodbc.connect(f'DRIVER={driver};SERVER={server};DATABASE={new_database_name};UID={username};PWD={password};TrustServerCertificate=yes')
cursor = conn.cursor()

In [28]:
get_current_database(cursor)

Currently connected to database: HospitalOperation


#### Load database configuration file from table_creation_code.txt

In [ ]:
def load_sql_from_txt(file_name: str, mode='r'):
    commands = []
    with open(file_name, mode) as file:
        imported_commands = file.read().split(';')
        for com in imported_commands:
            commands.append(com.replace('\n', ' ').strip())
    return commands




CREATE TABLE Physician (   EmployeeID INTEGER PRIMARY KEY NOT NULL,   Name TEXT NOT NULL,   Position TEXT NOT NULL,   SSN INTEGER NOT NULL )
CREATE TABLE Department (   DepartmentID INTEGER PRIMARY KEY NOT NULL,   Name TEXT NOT NULL,   Head INTEGER NOT NULL     CONSTRAINT fk_Physician_EmployeeID REFERENCES Physician(EmployeeID) )
CREATE TABLE Affiliated_With (   Physician INTEGER NOT NULL     REFERENCES Physician(EmployeeID),   Department INTEGER NOT NULL     REFERENCES Department(DepartmentID),   PrimaryAffiliation BIT NOT NULL,   PRIMARY KEY(Physician, Department) )
CREATE TABLE NewProcedure (   Code INTEGER PRIMARY KEY NOT NULL,   Name TEXT NOT NULL,   Cost REAL NOT NULL )
CREATE TABLE Trained_In (   Physician INTEGER NOT NULL      REFERENCES Physician(EmployeeID),   Treatment INTEGER NOT NULL    REFERENCES NewProcedure(Code),   CertificationDate DATETIME NOT NULL,   CertificationExpires DATETIME NOT NULL,   PRIMARY KEY(Physician, Treatment) )
CREATE TABLE Patient (   SSN INTEGER PR

In [ ]:
try:
    file_name = 'table_creation_code.txt'
    table_creation_codes = load_sql_from_txt()
    for command in table_creation_codes:
        if command: 
            cursor.execute(command)

    conn.commit()
    print('Database configurated succesfull from')

except Exception as e:
    print('Database configurated failed')


#### Load data from sample_dataset.txt

In [ ]:
try:
    file_name = 'sample_dataset.txt'
    data_codes = load_sql_from_txt()
    for command in data_codes:
        if command: 
            cursor.execute(command)

    conn.commit()
    print('Data loaded succesfully')

except Exception as e:
    print('Data loaded failed')
    print(e)


### **Query**

In [33]:
def query(sql_code, engine = conn):
    df_query = pd.read_sql(sql_code, engine)
    return df_query

In [ ]:
df

### **Close Connection**

In [ ]:
conn.close()